In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote as rcq
import calculate_trends as ct

In [7]:
roll_list = [5]
mom_list  = [5,15,30]
rsi_list  = [10,15,30]

In [8]:
tempDf = rcq.read_quote_data('IBM')

rcq.fill_quote_gaps( tempDf )
rcq.fix_splits( tempDf )

In [9]:
diffs = ct.generate_differentials(   tempDf                   ) # Differentials between c,ac, o, h, l, v
rolls = ct.generate_rolling_close(   tempDf,        roll_list ) # Rolling averages of adj c, listed as C_mean_n and C_std_10
moms  = ct.generate_momentum_close(  tempDf,         mom_list ) # Momentum between day and days offset in list, Momentum_n
rsis  = ct.generate_rsi(             tempDf, diffs,  rsi_list ) # Relative Strength Indexes over n days, RSI_n
bolls = ct.generate_bollinger_bands( tempDf, rolls, roll_list ) # percentB of Bollinger bands, 1 is at upper, 0 at lower

In [10]:
# Join everything, or can leave as seperate 
tempDf = tempDf.join( diffs, how='inner' )
tempDf = tempDf.join( rolls, how='inner' )
tempDf = tempDf.join(  moms, how='inner' )
tempDf = tempDf.join(  rsis, how='inner' )
tempDf = tempDf.drop( ['Open','High','Low','Volume','Close'], axis=1 )

In [11]:
tempDf.head()

,Adj Close,Diff_co,Diff_v,Close_mean_5,Close_std_5,Momentum_5,Momentum_15,Momentum_30,RSI_10,RSI_15,RSI_30
Date,,,,,,,,,,,
2017-03-01,181.949997,0.008145,-0.095493,180.834000,1.146965,0.004416,0.027618,0.092314,50.070191,50.030241,50.218339
2017-02-28,179.820007,0.002453,-0.108796,180.673999,0.998563,-0.002441,0.030603,0.083075,50.019917,49.976369,50.108271
2017-02-27,179.399994,-0.009879,0.123692,180.761997,0.920855,-0.007029,0.028430,0.076621,50.007195,49.915102,50.182521
2017-02-24,181.350006,0.006326,0.452033,181.015997,0.552884,-0.000441,0.046992,0.089621,49.915490,49.851535,50.157921
2017-02-23,181.649994,-0.001923,-0.253194,181.031995,0.565967,-0.000165,0.050469,0.106128,49.819682,49.756789,50.050590


In [8]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [63]:
print tempDf.shape[0]

3313


In [64]:
max_day = max( [max(roll_list),max(mom_list),max(rsi_list)] )
N_train = 2500
N_test  = 700
off     = 1

flipped_df = tempDf.copy()
flipped_df = flipped_df[::-1]
flipped_df = flipped_df[max_day:]

train_features = tempDf[:N_train]
train_labels   = tempDf.ix[off:N_train+off,'Adj Close'].values
test_features  = tempDf[N_train:N_test+N_train]

In [65]:
svr_clf = SVR()

svr_clf.fit( train_features, train_labels )

print svr_clf.score( train_features, train_labels )

0.879633070824
